In [2]:
# # To support both python 2 and python 3# To su 
# from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Manual RNN

  * Najpierw zaimplementujmy bardzo prosty model RNN, bez korzystania z żadnych operacji RNN TensorFlow, aby lepiej zrozumieć co dzieje się pod maską. 

  * Stworzymy RNN złożony z warstwy pięciu powtarzających się neuronów, wykorzystując funkcję aktywacji tanh. 
  
  * Przyjmujemy, że RNN przebiega tylko w dwóch etapach, pobierając wektory wejściowe o rozmiarze $3$ w każdym kroku czasowym. 

In [5]:
tf.reset_default_graph()

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()

Ta sieć wygląda podobnie jak dwuwarstwowa sieć neuronowa, z małymi modyfikacjami: 
  * po pierwsze te same wagi i bias są wspódzielone przez obie warstwy, 
  
  * po drugie przekaujemy dane wejściowe w każdej warstwie i otrzymujemy wyniki z każdej warstwy. 
  
Aby uruchomić model, musimy wrzucić dane do obu etapów czasowych:

In [6]:
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

  * Mini-batch zawiera cztery instancje, każda z sekwencją wejściowych złożona jest z dokładnie dwóch wejść.
  
  * Na końcu Y0_val i Y1_val zawierają dane wyjściowe sieci w obu etapach czasowych dla wszystkich neuronów i wszystkich instancji w mini-partii:

In [7]:
print(Y0_val)

[[ 0.33327058 -0.9674539   0.9919549  -0.9994455  -0.09661405]
 [ 0.9771993  -0.99999994  0.99999964 -1.         -0.5490335 ]
 [ 0.9994682  -1.          1.         -1.         -0.81342757]
 [ 0.5827164  -0.9999999  -0.9999993   0.9401039  -0.8222846 ]]


In [8]:
print(Y1_val)

[[ 0.9783739  -1.          1.         -1.         -0.97129875]
 [-0.9971186  -0.9792804   0.79844    -0.64475507 -0.16971755]
 [ 0.57230926 -1.          1.         -1.         -0.70548123]
 [ 0.9999481  -0.99691606  0.4274017  -0.9850408  -0.26429808]]


  * To nie było zbyt trudne, ale oczywiście, jeśli chcesz mieć możliwość uruchomienia RNN przez 100 iteracji to graf będzie dość duży. 
  * Teraz przyjrzyjmy się, jak stworzyć ten sam model za pomocą operacji RNN TensorFlow.

# Using static_rnn()

  * Funkcja **static_rnn()** tworzy rozwiniętą sieć RNN przez  łaczenie komórek. 
  * Poniższy kod tworzy dokładnie ten sam model, co poprzedni:

In [9]:
n_inputs = 3
n_neurons = 5

In [16]:
tf.reset_default_graph()

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1],
                                                dtype=tf.float32)
Y0, Y1 = output_seqs

  * Najpierw tworzymy wejściowe placeholdery zastępcze, tak jak poprzednio. 

  * Następnie tworzymy BasicRNNCell, który można uznać za fabrykę, która tworzy kopie komórki, aby zbudować rozwinięty RNN (po jednym dla każdego kroku czasowego). 
  * Następnie wywołujemy static_rnn(), podając mu:
    * fabrykę komórek 
    * tensory wejściowe
    * i informując o typie danych wejściowych (służy to do utworzenia macierzy stanu początkowego, która domyślnie jest pełna zer). 
  * Funkcja static_rnn() wywołuje funkcję fabryczną komórki raz na wejście, tworząc dwie kopie komórki (każda zawierająca warstwę pięciu powtarzających się neuronów), ze wspólnymi wagami oraz biasy, i łączy je tak jak wcześniej . 
  * Funkcja static_rnn() zwraca dwa obiekty. 
    * Pierwszym jest lista Pythona zawierająca tensory wyjściowe dla każdego kroku czasowego. 
    * Drugi to tensor zawierający ostatnie stany sieci. 
  * Kiedy używasz komórek podstawowych, stan końcowy jest po prostu równy ostatniemu wynikowi.

In [11]:
init = tf.global_variables_initializer()

In [12]:
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [14]:
print(Y0_val)

[[ 0.1376185   0.21949555 -0.61592793  0.56433624  0.65113115]
 [-0.5565757  -0.35894367 -0.9794814   0.9588838  -0.33179128]
 [-0.88409454 -0.7506628  -0.99909633  0.996841   -0.89899206]
 [-0.9975348  -0.6414948  -0.83871746  0.99752915 -0.9999351 ]]


In [13]:
print(Y1_val)

[[-0.92377996 -0.8825747  -0.99981666  0.98345184 -0.9992178 ]
 [ 0.35725626  0.58594644 -0.8181946  -0.55824023 -0.03909221]
 [-0.9304895  -0.67598426 -0.9986287   0.91753626 -0.9966239 ]
 [-0.7825352  -0.36849847 -0.96020097  0.42627504 -0.97051793]]


  * Załużmy, że chcemy zrobić 50 kroków. 
  * W takiej sytuacji nie byłoby zbyt łatwo zdefiniować 50 symboli wejściowych i 50 tensorów wyjściowych. 

  * Co więcej, w czasie wykonywania musiałbyśmy zainicjalizować każdy z 50 obiektów zastępczych i manipulować 50 wyjściami. 
  
  * Poniższy kod buduje ten sam RNN ponownie, ale tym razem wykorzystujemy jeden placeholder [Brak, n_steps, n_inputs], gdzie pierwszym wymiarem jest wielkością mini-bath. 
  
  * Następnie wyodrębnia listę sekwencji wejściowych dla każdego kroku czasowego. 
  
      * X_seqs powinien być lista n_steps tensowór o kształcie [None, n_inputs], w której pierwszym wymiarem jest rozmiarem mini-batch. 
  
      * Aby to zrobić, najpierw zamieniamy dwa pierwsze wymiary za pomocą funkcji transpose(), aby kroki czasowe były teraz pierwszym wymiarem. 
      
      * Następnie wyodrębniamy listę tensorów wzdłuż pierwszego wymiaru (tj. Jeden tensor na etap czasowy) za pomocą funkcji unstack().
      
  * Następne dwa wiersze są takie same jak wcześniej. 
  * Na koniec scalamy wszystkie tensory wyjściowe w jeden tensor za pomocą funkcji stack() aby otrzymać końcowy tensor kształtu [None, n_steps, n_neurons] (ponownie pierwszym wymiarem jest wielkością mini batha).

In [17]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [18]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs,
                                                dtype=tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

In [20]:
init = tf.global_variables_initializer()

Teraz możemy uruchomić sieć, podając jej pojedynczy tensor, który zawiera wszystkie sekwencje mini-batch:

In [21]:
X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

Otrzymujemy pojedynczy tensor **output_val** dla wszystkich instancji, wszystkich kroków czasowych i wszystkich neuronów:

In [28]:
print(outputs_val)

[[[-0.6649127   0.82071596 -0.48354113 -0.88919264 -0.7478687 ]
  [-0.98179144  0.9989492  -0.98389685  0.7443749  -0.99163437]]

 [[-0.9516823   0.99490875 -0.92497087 -0.93341523 -0.9742932 ]
  [-0.32576984 -0.77630436  0.5023754   0.44608352 -0.37081614]]

 [[-0.9939281   0.9998678  -0.99131024 -0.9603593  -0.997652  ]
  [-0.93323153  0.8879398  -0.8000195   0.8813868  -0.9528799 ]]

 [[-0.1395578   0.8392997   0.89317125  0.99993813 -0.8498562 ]
  [ 0.03385155 -0.07713976  0.35224912  0.7651471  -0.48652583]]]


# Funkcja dynamic_rnn()

  * Funkcja **dynamic_rnn()** używa operacji **while_loop()** do przepuszczenia przez neurony odpowiednią liczbę razy.

  * Ponadto możesz ustawić **swap_memory = True**, jeśli chcesz zamienić pamięć GPU na pamięć CPU podczas wstecznej propagacji, aby uniknąć błędów OOM. 

  * Akceptuje on również pojedynczy tensor dla wszystkich wejść w każdym kroku czasowym (kształt [Brak, n_steps, n_inputs]) i zwraca pojedynczy tensor dla wszystkich wyjść w każdym kroku czasowym (kształt [Brak, n_steps, n_neurons])

  * Nie ma potrzeby układania lub transpozycji. 

  * Poniższy kod tworzy ten sam RNN jak wcześniej przy użyciu funkcji dynamic_rnn (). Jest o wiele ładniej!

In [23]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [26]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

init = tf.global_variables_initializer()

In [27]:
X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [29]:
print(outputs_val)

[[[-0.6649127   0.82071596 -0.48354113 -0.88919264 -0.7478687 ]
  [-0.98179144  0.9989492  -0.98389685  0.7443749  -0.99163437]]

 [[-0.9516823   0.99490875 -0.92497087 -0.93341523 -0.9742932 ]
  [-0.32576984 -0.77630436  0.5023754   0.44608352 -0.37081614]]

 [[-0.9939281   0.9998678  -0.99131024 -0.9603593  -0.997652  ]
  [-0.93323153  0.8879398  -0.8000195   0.8813868  -0.9528799 ]]

 [[-0.1395578   0.8392997   0.89317125  0.99993813 -0.8498562 ]
  [ 0.03385155 -0.07713976  0.35224912  0.7651471  -0.48652583]]]


# Handling Variable Length Input Sequences

  * Do tej pory używaliśmy tylko stałych sekwencji wejściowych (wszystkie dokładnie o dwa kroki). 

  * Co jeśli sekwencje wejściowe mają zmienne długości (np. Podobne zdania)? 
  
  * W takim przypadku powinieneś ustawić parametr **sequence_length** przy wywołaniu funkcji **dynamic_rnn()** (lub static_rnn ()); 
  * musi to być tensor 1D, 
  * wskazuje on długość sekwencji wejściowej dla każdej instancji.

In [36]:
n_steps = 2
n_inputs = 3
n_neurons = 5

tf.reset_default_graph()

seq_length = tf.placeholder(tf.int32, [None])

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32,
sequence_length=seq_length)

In [39]:
X_batch = np.array([
    # step 0 step 1
    [[0, 1, 2], [9, 8, 7]], # instance 0
    [[3, 4, 5], [0, 0, 0]], # instance 1 (padded with a zero vector)
    [[6, 7, 8], [6, 5, 4]], # instance 2
    [[9, 0, 1], [3, 2, 1]], # instance 3
])
seq_length_batch = np.array([2, 1, 2, 2])
init = tf.global_variables_initializer()

In [40]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

In [41]:
print(outputs_val)

[[[-0.78141797 -0.49851418 -0.14493392 -0.92397135  0.89701474]
  [-0.94211817 -0.99670196  0.99974716 -1.          0.97302216]]

 [[-0.94907165 -0.93124336  0.8656041  -0.9999637   0.99156755]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.988933   -0.99245363  0.9922785  -1.          0.99933976]
  [-0.4879012  -0.9825125   0.99266934 -0.99978286  0.8595685 ]]

 [[ 0.9999413  -0.98737895  0.999641   -0.9742392  -0.9910776 ]
  [ 0.17476727 -0.40185434  0.9044615  -0.95206445  0.24472281]]]


In [42]:
print(states_val)

[[-0.94211817 -0.99670196  0.99974716 -1.          0.97302216]
 [-0.94907165 -0.93124336  0.8656041  -0.9999637   0.99156755]
 [-0.4879012  -0.9825125   0.99266934 -0.99978286  0.8595685 ]
 [ 0.17476727 -0.40185434  0.9044615  -0.95206445  0.24472281]]
